### Install required packages

In [ ]:
# pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

### Setup Langfuse Tracing

In [ ]:
from my_utils import setup_langfuse_tracer, hf_login
setup_langfuse_tracer()
hf_login()

### My LLM - To act as brain of all the below Agents

In [ ]:
from smolagents import LiteLLMModel
from my_config import MyConfig

my_config = MyConfig()

# remote_url = f"http://{my_config.OLLAMA_INFERENCE_NODE_IP}:11434"
remote_url = f"https://{my_config.OLLAMA_INFERENCE_RUNPOD_ID}-11434.proxy.runpod.net"
model = LiteLLMModel(
    base_url=remote_url
    , model_id='ollama/deepseek-coder-v2:16b'
    , max_tokens=8096
)
multimodal_model = LiteLLMModel(
    base_url=remote_url
    , model_id='ollama/gemma3:12b'
    , max_tokens=8096
)
manager_model = LiteLLMModel(
    base_url=remote_url
    # , model_id='ollama/deepseek-r1:14b'
    , model_id='ollama/qwen3:30b'
    , max_tokens=8096
)

In [ ]:
print(remote_url)

### Setting up single agent

In [ ]:
import os
# from langchain.agents import load_tools
from my_tools import calculate_cargo_travel_time
from smolagents import CodeAgent, GoogleSearchTool, VisitWebpageTool#, Tool

os.environ["SERPER_API_KEY"] = my_config.SERPER_API_KEY
os.environ["SERPAPI_API_KEY"] = my_config.SERPAPI_API_KEY

task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

# search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])
# Initialize the search tool
agent = CodeAgent(
    model=model,
    tools=[
        # search_tool
        GoogleSearchTool("serper")
        , VisitWebpageTool()
        , calculate_cargo_travel_time
    ],
    additional_authorized_imports=["pandas", "urllib.request", "bs4", "requests", "geopy.distance", "os"],
    max_steps=15,
)

In [ ]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

In [ ]:
detailed_report

### ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:

- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let’s create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to write its final report: so let us give it access to additional imports, including plotly, and geopandas + shapely for spatial plotting.

In [ ]:
import os
# from langchain.agents import load_tools
from my_tools import calculate_cargo_travel_time, calculate_car_travel_distance
from smolagents import CodeAgent, GoogleSearchTool, VisitWebpageTool, FinalAnswerTool

os.environ["SERPER_API_KEY"] = my_config.SERPER_API_KEY
os.environ["SERPAPI_API_KEY"] = my_config.SERPAPI_API_KEY

In [ ]:
web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
        calculate_car_travel_distance
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
        "requests",
        "bs4"
    ]
)

In [ ]:
web_agent.visualize()

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
# from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    # multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    # model=InferenceClientModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    model=manager_model,
    tools=[
        calculate_cargo_travel_time
        , calculate_car_travel_distance
        , FinalAnswerTool()
    ],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
        "requests",
        "bs4"
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

In [ ]:
manager_agent.visualize()

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

In [ ]:
manager_agent.run("""Hi, My name is John, I am looking for a holiday destination in Northern West Benagal, India. I am currently in Kolkata, India.
I prefer a quite hill station with lots of woods. I own a car, so I will driver there. I have a budget of 5000 INR per night for accomodation, food and sight-seeing.
I will keep the travel expense separate. Find me 3 good destination with their travel distance from Kolkata.
DO NOT USE GENERATE FACTS FROM EXISTING KNOWLEDGE BASE. DO NOT USE HYPOTHETICAL FACTS. CALCULATE DISTANCE USING PROVIDED TOOLS.
Also consider that currently petrol price is 100 INR per litre and my car gives 15 km per litre mileage. Then calculate the maximum expense for a 5 day trip.
In maximum expense you can consider the daily budget as daily expense.
Try to keep the calculation as accurate as possible, but don't worry about the exact number. Try to use the provided tools to find the information.
""")